In [ ]:
!pip install huggingface_hub
!pip install datasets
!pip install ipywidgets
!pip install pandas
!git clone https://github.com/Chris1nexus/community-events.git
%cd community-events
!pip install .
%cd ..
!rm -rf community-events
!pip install wandb
#!huggingface-cli login
#!wandb login
#!accelerate config

In [4]:
from sklearn.model_selection import ParameterGrid
from subprocess import call
import os


'''
Reference parameters 

hyperparams_cyclegan = {
    'decay_epoch':[10] , #100
    'lr':[0.0002], 
    'n_residual_blocks':[9], 
    
    'lambda_cyc':[10.0], 
    'lambda_id':[5.0], 
    'beta1':[0.5], 
    'beta2':[0.999], 
    'batch_size':[8],
    
    'dataset_name':['Chris1/sim2real_gta5_to_cityscapes'],#['Chris1/NFT_cryptopunks_to_bored_apes'],  

    'channels':[3], 
    'checkpoint_interval':[10], 
    #'cpu':[False],     
    'epoch':[0], 
    #'fp16':[False], 
    'image_size':[256], 
    'mixed_precision':['no'], 
    'num_epochs':[20], 
    'num_workers':[8], 
    'organization_name':['Chris1'], 
    
    'wandb':[True],
    'push_to_hub':[True], 
    'model_name':['test'],
    
    'pytorch_dump_folder_path':[None], 
    'sample_interval':[100]
                     }

hyperparams_cyclegan = { 
                        'lr':[0.0002], 
                        'num_epochs':[2], 
                        'decay_epoch':[1] , #100 
                        'n_residual_blocks':[9], 
                        'lambda_cyc':[10.0], 'lambda_id':[5.0], 
                        'beta1':[0.5], 'beta2':[0.999], 
                        'batch_size':[128],#[128], 
                        'dataset_name':['Chris1/NFT_cryptopunks_to_bored_apes_HQ'],#['Chris1/NFT_cryptopunks_to_bored_apes'], 
                        'channels':[3], 
                        'checkpoint_interval':[5], #'cpu':[False], 
                        'epoch':[0], #'fp16':[False], 
                        'image_size':[64], #256
                        'mixed_precision':['no'], 
                        'num_workers':[8], 
                        'organization_name':['Chris1'], 
                        'wandb':[True],
                        'push_to_hub':[True], 
                        'model_name':['test_cyclegan_punk_to_apes'],
                        'pytorch_dump_folder_path':['torch_dump_punk_to_apes'], 
                        'sample_interval':[30] }
'''
hyperparams_cyclegan = { 
                        'lr':[0.0002], 
                        'num_epochs':[200], 
                        'decay_epoch':[100] , #100 
                        'n_residual_blocks':[9], 
                        'lambda_cyc':[10.0], 
                        'lambda_id':[5.0], 
                        'beta1':[0.5], 'beta2':[0.999], 
                        'batch_size':[8], 
                        'source_dataset_name':['huggingnft/cryptoadz-by-gremplin'],
                        'target_dataset_name':['huggingnft/mini-mutants'],
                        #'dataset_name':['Chris1/dooggies__2__nftrex'],#['Chris1/NFT_cryptopunks_to_bored_apes_HQ'],#['Chris1/NFT_cryptopunks_to_bored_apes'], 
                        'channels':[3], 
                        'checkpoint_interval':[5], #'cpu':[False], 
                        'epoch':[0], #'fp16':[False], 
                        'image_size':[256], #256
                        'mixed_precision':['no'], 
                        'num_workers':[8], 
                        'organization_name':['Chris1'], 
                        'wandb':[True],
                        'push_to_hub':[True], 
                        'model_name':['test_cyclegan_toadz__2__minimutants'],
                        'pytorch_dump_folder_path':['torch_dump_toadz__2__minimutants'], 
                        'sample_interval':[30] }

In [38]:
import copy
import pandas as pd
EXPERIMENTS_DATA_CSV = 'experiments.csv'

if not os.path.exists(EXPERIMENTS_DATA_CSV):
    experiments_df = pd.DataFrame()
else:
    experiments_df = pd.read_csv(EXPERIMENTS_DATA_CSV)


EXPERIMENT_NAME = 'cyclegan_toadz__2__minimutants'
    

for idx, grid in enumerate(ParameterGrid(hyperparams_cyclegan)):

        copy_grid = copy.deepcopy(grid)
        if 'dataset_name' in copy_grid:
            del copy_grid['dataset_name']
        if 'dataset' in copy_grid:
            del copy_grid['dataset']
        if 'output_dir' in copy_grid:
            del copy_grid['output_dir']
            
        experiment_name =  '--'.join([f'{k}__{v}'  for k,v in copy_grid.items() ])
        call_params = ['accelerate', 'launch',
                       '--config_file','~/.cache/huggingface/accelerate/default_config.yaml',
                       'train.py']
        
        
        experiment_id = len(experiments_df)
        grid['output_dir'] = experiment_name
        
        experiments_df = experiments_df.append(grid,ignore_index=True)
        grid['output_dir'] = f'experiments/{EXPERIMENT_NAME}/{experiment_id}'
        grid['pytorch_dump_folder_path'] = f'experiments/{grid["pytorch_dump_folder_path"]}_{experiment_id}'
        grid['model_name'] = f'{grid["model_name"]}_{experiment_id}'
        for k,v in grid.items():
            
             
            call_params.append(f'--{k}' )
            if k not in  ['cpu', 'wandb','fp16','push_to_hub']:
                call_params.append(f'{v}' )
                
        print(' '.join(call_params))
        call(call_params)
        
        
        experiments_df.to_csv(EXPERIMENTS_DATA_CSV)

/tmp/ipykernel_8531/2398649201.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  experiments_df = experiments_df.append(grid,ignore_index=True)


accelerate launch --config_file ~/.cache/huggingface/accelerate/default_config.yaml train.py --batch_size 8 --beta1 0.5 --beta2 0.999 --channels 3 --checkpoint_interval 5 --decay_epoch 100 --epoch 0 --image_size 256 --lambda_cyc 10.0 --lambda_id 5.0 --lr 0.0002 --mixed_precision no --model_name test_cyclegan_toadz__2__minimutants_3 --n_residual_blocks 9 --num_epochs 200 --num_workers 8 --organization_name Chris1 --push_to_hub --pytorch_dump_folder_path experiments/torch_dump_toadz__2__minimutants_3 --sample_interval 30 --source_dataset_name huggingnft/cryptoadz-by-gremplin --target_dataset_name huggingnft/mini-mutants --wandb --output_dir experiments/cyclegan_toadz__2__minimutants/3
Namespace(batch_size=8, beta1=0.5, beta2=0.999, channels=3, checkpoint_interval=5, cpu=False, decay_epoch=100, epoch=0, fp16=False, image_size=256, lambda_cyc=10.0, lambda_id=5.0, lr=0.0002, mixed_precision='no', model_name='test_cyclegan_toadz__2__minimutants_3', n_residual_blocks=9, num_epochs=200, num_wo

KeyboardInterrupt: 

In [36]:
!accelerate launch --config_file ~/.cache/huggingface/accelerate/default_config.yaml train.py --batch_size 8 --beta1 0.5 --beta2 0.999 --channels 3 --checkpoint_interval 5 --decay_epoch 100 --epoch 0 --image_size 256 --lambda_cyc 10.0 --lambda_id 5.0 --lr 0.0002 --mixed_precision no --model_name test_cyclegan_toadz__2__minimutants_2 --n_residual_blocks 9 --num_epochs 200 --num_workers 8 --organization_name Chris1 --push_to_hub --pytorch_dump_folder_path experiments/torch_dump_toadz__2__minimutants_2 --sample_interval 30 --source_dataset_name Chris1/cryptopunks_HQ --target_dataset_name Chris1/bored_apes_yacht_club_HQ --wandb --output_dir experiments/cyclegan_toadz__2__minimutants/2

/home/xrh1/.local/lib/python3.8/site-packages/torch/distributed/launch.py:178: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
Namespace(batch_size=8, beta1=0.5, beta2=0.999, channels=3, checkpoint_interval=5, cpu=False, decay_epoch=100, epoch=0, fp16=False, image_size=256, lambda_cyc=10.0, lambda_id=5.0, lr=0.0002, mixed_precision='no', model_name='test

Traceback (most recent call last):
  File "/home/xrh1/.local/bin/accelerate", line 8, in <module>
    sys.exit(main())
  File "/home/xrh1/.local/lib/python3.8/site-packages/accelerate/commands/accelerate_cli.py", line 43, in main
    args.func(args)
  File "/home/xrh1/.local/lib/python3.8/site-packages/accelerate/commands/launch.py", line 468, in launch_command
    multi_gpu_launcher(args)
  File "/home/xrh1/.local/lib/python3.8/site-packages/accelerate/commands/launch.py", line 228, in multi_gpu_launcher
    raise subprocess.CalledProcessError(returncode=process.returncode, cmd=cmd)
subprocess.CalledProcessError: Command '['/usr/bin/python3', '-m', 'torch.distributed.launch', '--use_env', '--nproc_per_node', '2', 'train.py', '--batch_size', '8', '--beta1', '0.5', '--beta2', '0.999', '--channels', '3', '--checkpoint_interval', '5', '--decay_epoch', '100', '--epoch', '0', '--image_size', '256', '--lambda_cyc', '10.0', '--lambda_id', '5.0', '--lr', '0.0002', '--mixed_precision', 'no', '-

In [ ]:
!accelerate launch --config_file ~/.cache/huggingface/accelerate/default_config.yaml train.py --batch_size 8 --beta1 0.5 --beta2 0.999 --channels 3 --checkpoint_interval 5 --decay_epoch 100 --epoch 0 --image_size 256 --lambda_cyc 10.0 --lambda_id 5.0 --lr 0.0002 --mixed_precision no --model_name test_cyclegan_toadz__2__minimutants_3 --n_residual_blocks 9 --num_epochs 200 --num_workers 8 --organization_name Chris1 --push_to_hub --pytorch_dump_folder_path experiments/torch_dump_toadz__2__minimutants_3 --sample_interval 30 --source_dataset_name huggingnft/cryptoadz-by-gremplin --target_dataset_name huggingnft/mini-mutants --wandb --output_dir experiments/cyclegan_toadz__2__minimutants/3

/home/xrh1/.local/lib/python3.8/site-packages/torch/distributed/launch.py:178: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
Namespace(batch_size=8, beta1=0.5, beta2=0.999, channels=3, checkpoint_interval=5, cpu=False, decay_epoch=100, epoch=0, fp16=False, image_size=256, lambda_cyc=10.0, lambda_id=5.0, lr=0.0002, mixed_precision='no', model_name='test